## Libraries

In [47]:
from gplearn.genetic import SymbolicRegressor
from gplearn.fitness import make_fitness
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [88]:
# Plotly
import chart_studio.plotly as ply
import cufflinks as cf
cf.go_offline()

In [134]:
from scipy import signal

## Functions

In [48]:
def calor():
    # Inicializar posicion de datos variables difusas

    calor1 = [9, 151, 365.25+9]
    calor2 = [9-365.25, 151-365.25, 9]
    frio1 = [151-365.25, 9, 151]
    frio2 = [151, 365.25+9, 365.25+365.25]

    height = {'calor1': calor1, 'calor2': calor2, 'frio1': frio1, 'frio2': frio2} 
    return height

# para crear variables difusas
def triangular1():
    
    # Se crea x
    ma = pd.DataFrame();
    x = consumofeb.index.dayofyear.to_list()
    # Se crean variables a, b y c
    height = calor()
    for hei in height:
        ba = pd.DataFrame(x, columns=["x"])
        ba["a"] = height[hei][0]
        ba["b"] = height[hei][1]
        ba["c"] = height[hei][2]
        # Variable difusa
        ca = pd.DataFrame()
        ca["bo"] = (ba.x-ba.a)/(ba.b-ba.a)
        ca["ba"] = (ba.c-ba.x)/(ba.c-ba.b)
        ca = ca.min(axis = 1)
        ca[ca<0]=0
        ma = pd.concat([ma, ca], axis = 1) # Se concatenan

    ma.index = consumofeb.index
    ma.columns = height.keys()
    return ma

In [49]:
def difs():
    # Inicializar posicion de datos variables difusas

    inv1 = [310.5-365.25, 8.6529, 138.941176470588]
    pr = [8.6529, 138.941176470588, 221.9375]
    vr = [138.941176470588, 221.9375, 310.5]
    ot = [221.9375, 310.5, 365.25+8.6529]
    ot2 = [221.9375-365.25, 310.5-365.25, 8.6529]
    inv2 = [310.5, 365.25+8.6529, 365.25+138.941176470588]

    height = {'Invierno1': inv1,'Primavera': pr,'verano': vr,'Otoño': ot,'Otoño2': ot2,'Invierno2': inv2,} 
    return height 

# para crear variables difusas
def triangular():
    # Se crea x
    ma = pd.DataFrame();
    x = consumofeb.index.dayofyear.tolist()
    # Se crean variables a, b y c
    height = difs()
    for hei in height:
        ba = pd.DataFrame(x, columns=["x"])
        ba["a"] = height[hei][0]
        ba["b"] = height[hei][1]
        ba["c"] = height[hei][2]
        # Variable difusa
        ca = pd.DataFrame()
        ca["bo"] = (ba.x-ba.a)/(ba.b-ba.a)
        ca["ba"] = (ba.c-ba.x)/(ba.c-ba.b)
        ca = ca.min(axis = 1)
        ca[ca<0]=0
        ma = pd.concat([ma, ca], axis = 1) # Se concatenan
        
    ma.index = consumofeb.index
    ma.columns = height.keys()
    return ma

In [50]:
# custom metric
def _mape(y, y_pred, w):
    diffs = np.abs(np.divide((np.maximum(0.001, y) - np.maximum(0.001, y_pred)),np.maximum(0.001, y)))
    
    return 100. * np.average(diffs, weights=w)

mape = make_fitness(_mape, greater_is_better=False)

## seasonal decompose probe

## Load data

In [51]:
path_consumofeb = Path().resolve().parents[1] / "Data" / "Consumo_feb.xlsx"
consumofeb=pd.read_excel(path_consumofeb).set_index("fecha").loc["2007-01-01":"2020-03-30"] # desde el 2007

In [150]:
def independent():
    # date dependent variables
    fecha = pd.DataFrame(consumofeb.index.year.values, index = consumofeb.index, columns = ["year"])
    fecha.year = fecha.year/fecha.year.max()
    fecha["week"] = consumofeb.index.week/53
    fecha["day"] = consumofeb.index.day/31
    # fecha = pd.get_dummies(fecha, columns = ["year", "week"], prefix = ["year", "week"], drop_first = True)
    triangular_ = pd.concat([triangular(), triangular1()], axis = 1)
    X = pd.concat([fecha, triangular_, sencos], axis = 1)
    return X
X1 =  independent()

C:\Users\alber\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\alber\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:21: FutureWarning:

weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)



In [151]:
X_train, X_test, y_train, y_test = train_test_split(X1.to_numpy(), consumofeb["max"].to_numpy(), test_size = 365, shuffle = False)

## Fit the model

In [152]:
# ?SymbolicRegressor

In [154]:
functions=('add', 'sub', 'mul', 'div', 'sin', 'cos', 'abs')
model = SymbolicRegressor(population_size = 10000, generations = 20, const_range = (-10, 10), function_set = functions,
                metric = mape, parsimony_coefficient = 0.003, feature_names = [*X1.columns],
                p_crossover = 0.7, p_subtree_mutation = 0.2, p_hoist_mutation = 0.03, max_samples = 0.9,
                verbose = 1)

In [155]:
model.fit(X_train, y_train)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.06          108.848       37          40.9854          42.2968      6.30m
   1    13.48          94.2732       39          20.1302          14.6836     11.01m
   2    16.29            84.69        7          8.65489          7.88563     12.96m
   3    24.14          78.1464       20          7.22638          7.11846     10.52m
   4    25.88          66.8811       47          6.87606          6.81168     10.17m
   5    25.06          58.8378       17          6.21773          6.08984      9.36m
   6    24.06          34.8786       51          5.67094          5.70737      8.00m
   7    24.76          33.8473       47           5.4837          5.96954      7.56m
   8    28.44          32.4665       49          5.35126          5.59487  

SymbolicRegressor(const_range=(-10, 10),
                  feature_names=['year', 'week', 'day', 'Invierno1',
                                 'Primavera', 'verano', 'Otoño', 'Otoño2',
                                 'Invierno2', 'calor1', 'calor2', 'frio1',
                                 'frio2', 't', '179.19_sen', '179.19_cos',
                                 '345.57_sen', '345.57_cos', '186.08_sen',
                                 '186.08_cos', '4838.00_sen', '4838.00_cos',
                                 '403.17_sen', '403.17_cos', '604.75_sen',
                                 '604.75_cos', '322.53_sen', '322....
                  init_method='half and half', low_memory=False,
                  max_samples=0.9,
                  metric=<gplearn.fitness._Fitness object at 0x000002378428C908>,
                  n_jobs=1, p_crossover=0.7, p_hoist_mutation=0.03,
                  p_point_mutation=0.01, p_point_replace=0.05,
                  p_subtree_mutation=0.2, parsimony_coe

In [92]:
model.fit(X_train, y_train)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.36          101.971       11          27.1611          25.8414      5.92m
   1    15.26           93.429        7          11.4823           11.801      6.96m
   2    14.80          80.2309        7          5.42954          5.64885      6.41m
   3    18.58          69.2494        7          5.47173          5.26987      6.45m
   4    22.15          62.8202        7          5.45369          5.43195      6.47m
   5    22.09          45.3482        7          5.42473          5.69211      5.80m
   6    23.97          38.3968        7          5.39845          5.92823      5.55m
   7    24.96          41.2006       20          5.08706          5.05095      5.17m
   8    26.29          42.0589       20          5.03315           5.5353  

SymbolicRegressor(const_range=(-10, 10),
                  feature_names=['year', 'week', 'day', 'Invierno1',
                                 'Primavera', 'verano', 'Otoño', 'Otoño2',
                                 'Invierno2', 'calor1', 'calor2', 'frio1',
                                 'frio2'],
                  function_set=('add', 'sub', 'mul', 'div', 'sin', 'cos',
                                'abs'),
                  generations=20, init_depth=(2, 6),
                  init_method='half and half', low_memory=False,
                  max_samples=0.9,
                  metric=<gplearn.fitness._Fitness object at 0x000002378428C908>,
                  n_jobs=1, p_crossover=0.7, p_hoist_mutation=0.03,
                  p_point_mutation=0.01, p_point_replace=0.05,
                  p_subtree_mutation=0.2, parsimony_coefficient=0.003,
                  population_size=10000, random_state=None,
                  stopping_criteria=0.0, tournament_size=20, verbose=1,
              

In [162]:
predict = model.predict(X_test)

In [163]:
plotting = pd.DataFrame(predict, index = consumofeb.head(365).index, columns = ["pred"])
plotting["real"] = y_test

In [164]:
# plotting = pd.DataFrame(predict, columns = ["pred"])
# plotting["real"] = y_train

In [165]:
plotting.iplot()

In [166]:
np.mean(np.abs((plotting.real - plotting.pred)/plotting.real))*100

5.352272822735782

In [130]:
np.mean(np.abs((plotting.real - plotting.pred)/plotting.real))*100

4.925880880328518

In [161]:
print(model._program)

add(add(abs(sub(div(abs(sin(calor1)), sin(sin(-6.438))), abs(add(add(Primavera, year), cos(t))))), add(add(add(sin(Otoño2), cos(sub(322.53_sen, 439.82_cos))), add(add(add(Primavera, year), cos(t)), add(cos(cos(Otoño2)), abs(-9.374)))), abs(-9.374))), cos(1612.67_sen))


abs(sub(mul(sin(calor1), 8.927), add(cos(sub(mul(2.555, abs(verano)), calor1)), add(div(verano, mul(week, week)), div(div(abs(cos(mul(sin(sin(verano)), Primavera))), -0.041), sin(sub(-8.017, 9.572)))))))

add(mul(add(sub(calor1, -4.543), mul(sub(Invierno1, verano), Primavera)), 5.528), sub(Primavera, Invierno1))

In [ ]:
np.add(np.multiply(np.add(.np.sub(calor1, -4.543), np.multiply(np.sub(Invierno1, verano), Primavera)), 5.528), np.sub(Primavera, Invierno1))